### find sensors with upper depth not responding (installed higher than ground level)

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time
import datetime as dt
import re

from IPython.display import display, HTML, clear_output
from IPython.core.interactiveshell import InteractiveShell
import ipywidgets as widgets

from matplotlib import pyplot as plt
import seaborn as sns

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:79% !important; }</style>"))

# easier-to-read notebook:
from IPython.display import display, HTML, clear_output
pd.options.display.max_columns = 70
pd.options.display.max_rows = 500
%load_ext autoreload
%autoreload 2

r = re.compile('.*maoz*.')
if len(list(filter(r.match, sys.path))) != 0:
    PHYTECH_DRIVE_PATH = os.environ['PHYTECH_DRIVE_PATH']
    #PHYTECH_DRIVE_PATH = '/Users/maozdotan/Google Drive/My Drive'
    GITHUB_PATH = PHYTECH_DRIVE_PATH + '/GitHub'
    DATA_WD = PHYTECH_DRIVE_PATH + '/Data Integrity/SM anomalies/data'
    CERT_PATH = PHYTECH_DRIVE_PATH + '/Data'
    if CERT_PATH not in sys.path:
        sys.path.append(CERT_PATH)
    from sql_import_export import SqlImporter
    import cert_aws as c
    sql_importer = SqlImporter(database = c.database_research, user = c.user_research, password = c.password_research,
                                host = c.host_research, port = c.port_research, verbose=True)

local aws_cert.py loaded


In [3]:
%reload_ext autoreload
import project_class_data_extract
import logic_parameters# import default_latitude, default_height
from tqdm import tqdm_notebook
from functions import find_not_responding_events,get_project_results,load_project_data,not_responding_logic
#import common_db as cdb

In [4]:
summary_df = pd.read_csv(DATA_WD+'/projects_df_2023_05_15 15-08-24.csv').iloc[:,1:]
washington = summary_df[summary_df.territory == 'washington']
washington

,project_id,sensor_id,area_id,area_name,company_name,crop_name,variety_id,probe_depths,irrigation_events,not_responding_events_count,max_SM_diff,SM_statistics,event_timestamp,support_status,support_updated_at,days_since_task_complete,remarks,link,timezone,territory,percent_not responding
20,850367,1149114,12935,Section 12,8678,apple,21,"[16.0, 31.0]",1,0,12.76,"{16.0: {'mean': 33.87623036649215, 'std': 2.32...",set(),completed,2023-04-29 03:10:22.091162,16.0,set(),https://app.phytech.com/12935/46900/850367,America/Los_Angeles,washington,0.0
22,850457,1117648,8291,Cherry Ridge Farms,8181,cherry,22,"[16.0, 31.0]",5,1,0.88,"{16.0: {'mean': 18.1020320855615, 'std': 0.509...","{Timestamp('2023-05-12 09:46:00'), Timestamp('...",NaN,NaN,NaN,{'2023-05-08 07:09:00: probe 31.0 Not respondi...,https://app.phytech.com/8291/38806/850457,America/Los_Angeles,washington,0.2
23,850570,1117596,11137,Colt- Jeff,7975,apple,21,[23.0],1,0,2.23,"{23.0: {'mean': 34.682239583333335, 'std': 0.4...",set(),completed,2023-03-14 03:10:19.265450,62.0,set(),https://app.phytech.com/11137/55249/850570,America/Los_Angeles,washington,0.0
24,850573,1125822,11137,Colt- Jeff,7975,apple,21,"[10.0, 25.0]",1,0,2.15,"{10.0: {'mean': nan, 'std': nan, 'max': nan, '...",set(),new,2023-04-30 23:39:37.710479,15.0,{'missing SM data for 10.0'},https://app.phytech.com/11137/55250/850573,America/Los_Angeles,washington,0.0
25,850665,1140618,8800,CPC - Cardinal Crest,9353,apple,21,"[16.0, 31.0]",5,0,3.35,"{16.0: {'mean': 14.190847457627116, 'std': 1.2...",set(),completed,2023-05-07 03:10:16.980802,8.0,set(),https://app.phytech.com/8800/41424/850665,America/Los_Angeles,washington,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,876765,1148058,19215,WHITSTRAN,7975,apple,144,"[15.0, 30.0]",3,0,0.94,"{15.0: {'mean': 28.457135416666663, 'std': 0.9...",{Timestamp('2023-05-12 08:10:00')},completed,2023-04-19 00:16:13.908251,26.0,{'2023-05-12 08:10:00: probe 30.0 Low sensor r...,https://app.phytech.com/19215/84607/876765,America/Los_Angeles,washington,0.0
2084,876766,1148055,19215,WHITSTRAN,7975,apple,143,"[16.0, 31.0]",3,0,10.18,"{16.0: {'mean': 29.74677083333334, 'std': 3.57...",set(),NaN,NaN,NaN,set(),https://app.phytech.com/19215/84609/876766,America/Los_Angeles,washington,0.0
2085,876768,1147999,19215,WHITSTRAN,7975,apple,143,"[16.0, 31.0]",2,0,11.50,"{16.0: {'mean': nan, 'std': nan, 'max': nan, '...",set(),NaN,NaN,NaN,{'missing SM data for 16.0'},https://app.phytech.com/19215/84611/876768,America/Los_Angeles,washington,0.0
2090,876962,1120686,9467,Gap Ranch,19740,blueberry,19,"[15.0, 30.0]",0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://app.phytech.com/9467/85309/876962,America/Los_Angeles,washington,0.0
